# SQLite3  
## A DB-API for SQLite database

SQLite is a C based library that provides a light-weighted disk based database that doesn't require a 
seperate server process that allow access to database. It's built-in library in python, The sqlite3 module was written by Gerhard Häring. It provides a SQL interface compliant with the DB-API 2.0 specification described by <a href = "https://www.python.org/dev/peps/pep-0249">PEP 249</a>, and requires SQLite 3.7.15 or newer.

It's possible to create our prototypes with SQLite3 as our base db and later we can port to larger db's like PostgreSQl/Oracle.

# Connection API

In [2]:
#To use the module, you must first create a Connection object that 
# represents the database. Here the data will be stored in the example.db file:
import sqlite3
con = sqlite3.connect('expl.db')
# You can also give (:memory:) to create in-ram db

 Once you created connection, next step is to create cursor object where we can call its execute
 method to perform SQL commands.

 Here , Cursor is like a physical work-area or row pointing our table inside database. So that when we execute any command it will retrieve data row wise.

In [3]:
cur = con.cursor()

# Create table
cur.execute('''CREATE TABLE stocks
               (date text, trans text, symbol text, qty real, price real)''')

# Insert a row of data
cur.execute("INSERT INTO stocks VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Save (commit) the changes
con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
# con.close()

The Data which we have saved in the memory or in the file is persistant and it can be available any time when you retrieve them.

In [4]:
for row in cur.execute('SELECT * FROM stocks ORDER BY price'):
    print(row)

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


In [5]:
# Basic db api for creating and retrieving data 

connection = sqlite3.connect(":memory:")
cursr = connection.cursor()
cursr.execute("create table lang (name, first_appeared)")

# This is the qmark style:
cursr.execute("insert into lang values (?, ?)", ("C", 1972))

# The qmark style used with executemany():
lang_list = [
    ("Fortran", 1957),
    ("Python", 1991),
    ("Go", 2009),
]
cursr.executemany("insert into lang values (?, ?)", lang_list)

# And this is the named style:
cursr.execute("select * from lang where first_appeared=:year", {"year": 1972})
print(cursr.fetchone())
print(cursr.fetchall())

connection.close()

('C', 1972)
[]


# SQL and Python Types
***
The following Python types can thus be sent to SQLite
<div align="center">

| Python | SQLite3 |
|-----|--------|
| None | NULL |
| int | INTEGER |
| float | REAL |
| str | TEXT |
| bytes | BLOB |

</div>

*** 

SQLite3 System much extensible, as we can even store additional python types using adaption , and let sqlite to convert using convertors. 

As described before, SQLite supports only a limited set of types natively. To use other Python types with SQLite, you must adapt them to one of the sqlite3 module’s supported types for SQLite: one of NoneType, int, float, str, bytes.

There are two ways to enable the sqlite3 module to adapt a custom Python type to one of the supported ones.

In [6]:
# Letting your object adapt itself 

class Point:
    def __init__(self, x, y):
        self.x, self.y = x, y

    def __conform__(self, protocol):
        if protocol is sqlite3.PrepareProtocol:
            return "%f;%f" % (self.x, self.y)

con = sqlite3.connect(":memory:")
cur = con.cursor()

p = Point(4.0, -3.2)
cur.execute("select ?", (p,))
print(cur.fetchone()[0])

con.close()

4.000000;-3.200000


In [7]:
# Registering an adapter callable
# The other possibility is to create a function that converts the type to the 
# string representation and register the function with register_adapter().

class Pnt:
    def __init__(self, x, y):
        self.x, self.y = x, y

def adapt_point(point):
    return "%f;%f" % (point.x, point.y)

sqlite3.register_adapter(Pnt, adapt_point)

con = sqlite3.connect(":memory:")
cur = con.cursor()

p = Pnt(4.0, -3.2)
cur.execute("select ?", (p,))
print(cur.fetchone()[0])

con.close()

4.000000;-3.200000


## Some Shortcut Methods

In [8]:
#Using the nonstandard execute(), executemany() and executescript() methods of the 
# Connection object, your code can be written more concisely because you don’t have to 
# create the (often superfluous) Cursor objects explicitly. Instead, 
# the Cursor objects are created implicitly and these shortcut methods return the cursor objects.

langs = [
    ("C++", 1985),
    ("Objective-C", 1984),
]

con = sqlite3.connect(":memory:")

# Create the table
con.execute("create table lang(name, first_appeared)")

# Fill the table
con.executemany("insert into lang(name, first_appeared) values (?, ?)", langs)

# Print the table contents
for row in con.execute("select name, first_appeared from lang"):
    print(row)

print("I just deleted", con.execute("delete from lang").rowcount, "rows")

# close is not a shortcut method and it's not called automatically,
# so the connection object should be closed manually
con.close()

('C++', 1985)
('Objective-C', 1984)
I just deleted 2 rows


In [9]:
# Accessing columns by names 
con = sqlite3.connect(":memory:")
con.row_factory = sqlite3.Row

cur = con.cursor()
cur.execute("select 'John' as name, 42 as age")
for row in cur:
    assert row[0] == row["name"]
    assert row["name"] == row["nAmE"]
    assert row[1] == row["age"]
    assert row[1] == row["AgE"]

con.close()

# CRUD OPERATIONS 

In [13]:
# creating,read,update, delete table in database

#connect to sqlite db
conn = sqlite3.connect(":memory:")
#create a cursor obj
cur = conn.cursor()
# drop query   
cur.execute("DROP TABLE IF EXISTS STUDENT")

# create query 
query = '''CREATE TABLE STUDENT(
    ID INT PRIMARY KEY NOT NULL,
    NAME CHAR(20) NOT NULL,
    ROLL CHAR(20),
    ADDRESS CHAR(50), 
    CLASS CHAR(20)
)'''
cur.execute(query)
conn.commit()

#static insertion
conn.execute("INSERT INTO STUDENT (ID,NAME,ROLL,ADDRESS,CLASS) "
             "VALUES (1, 'John', '001', 'Bangalore', '10th')")
conn.execute("INSERT INTO STUDENT (ID,NAME,ROLL,ADDRESS,CLASS) "
             "VALUES (2, 'Naren', '002', 'Hyd', '12th')")
conn.commit()
# param/args insert
query = ('INSERT INTO STUDENT (ID,NAME,ROLL,ADDRESS,CLASS) '
         'VALUES (:ID, :NAME, :ROLL, :ADDRESS, :CLASS);')
params = {
        'ID': 3,
        'NAME': 'Jax',
        'ROLL': '003',
        'ADDRESS': 'Delhi',
        'CLASS': '9th'
    }
conn.execute(query, params)
conn.commit()

cur.execute('SELECT * FROM STUDENT')
print(cur.fetchall())

cur.execute("UPDATE STUDENT set ROLL = 005 where ID = 1")
cur.execute('SELECT * FROM STUDENT')
print(cur.fetchall())

conn.close()

[(1, 'John', '001', 'Bangalore', '10th'), (2, 'Naren', '002', 'Hyd', '12th'), (3, 'Jax', '003', 'Delhi', '9th')]
[(1, 'John', '5', 'Bangalore', '10th'), (2, 'Naren', '002', 'Hyd', '12th'), (3, 'Jax', '003', 'Delhi', '9th')]


Apart from creating our own statements to read and print to just alone work with sql files generated we can use the <a href="https://sqliteonline.com/" alt="sql_online_interpretor">SQLiteonline</a> site.